# VaR Analysis for Kenyan Banks - Financial Risk Management Project


Data Setup, Cleaning, and Preprocessing

This section prepares the financial time-series data for the Value-at-Risk (VaR) analysis.  
We load historical price data for **KCB**, **Equity Bank**, and **Co-operative Bank**, then clean and format the datasets to ensure consistency.

Key Steps:
- Load CSV files for the three Kenyan banks.
- Strip whitespace from column names (e.g., `" Close"` → `"Close"`).
- Convert the `Date` column into a proper datetime format.
- Sort the data chronologically to prepare for returns calculations.

In [2]:
# VaR Analysis for Kenyan Banks - Financial Risk Management Project
# Part 1: Setup, Data Loading, and Preprocessing




import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print(" All libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")




# Load the three bank datasets
kcb_data = pd.read_csv('kcb_historical_prices.csv')
equity_data = pd.read_csv('equity_historical_prices.csv')
coop_data = pd.read_csv('coop_historical_prices.csv')

# Convert Date columns to datetime
kcb_data['Date'] = pd.to_datetime(kcb_data['Date'])
equity_data['Date'] = pd.to_datetime(equity_data['Date'])
coop_data['Date'] = pd.to_datetime(coop_data['Date'])

# Sort by date
kcb_data = kcb_data.sort_values('Date').reset_index(drop=True)
equity_data = equity_data.sort_values('Date').reset_index(drop=True)
coop_data = coop_data.sort_values('Date').reset_index(drop=True)

print("=" * 70)
print("STOCK PRICE DATA LOADED")
print("=" * 70)
print(f"\nKCB Data Shape: {kcb_data.shape}")
print(f"Date Range: {kcb_data['Date'].min()} to {kcb_data['Date'].max()}")
print(f"\nFirst 5 rows:")
print(kcb_data.head())

print(f"\n\nEquity Bank Data Shape: {equity_data.shape}")
print(f"Date Range: {equity_data['Date'].min()} to {equity_data['Date'].max()}")

print(f"\n\nCo-op Bank Data Shape: {coop_data.shape}")
print(f"Date Range: {coop_data['Date'].min()} to {coop_data['Date'].max()}")




# Load interest rate data
interest_rates = pd.read_csv('central_bank_rates.csv')

# Handle missing values with median imputation
print("\n" + "=" * 70)
print("INTEREST RATE DATA")
print("=" * 70)
print(f"\nOriginal shape: {interest_rates.shape}")
print(f"Missing values before imputation:")
print(interest_rates.isnull().sum())

# Impute missing values with median
for col in interest_rates.columns:
    if interest_rates[col].dtype in ['float64', 'int64']:
        interest_rates[col].fillna(interest_rates[col].median(), inplace=True)

print(f"\nMissing values after imputation:")
print(interest_rates.isnull().sum())
print(f"\nSample of interest rate data:")
print(interest_rates.head())

# Load USD/KES exchange rate data
usd_kes_data = pd.read_csv('USD_KES_historical_data.csv')
usd_kes_data['Date'] = pd.to_datetime(usd_kes_data['Date'])
usd_kes_data = usd_kes_data.sort_values('Date').reset_index(drop=True)

print("\n" + "=" * 70)
print("USD/KES EXCHANGE RATE DATA")
print("=" * 70)
print(f"Shape: {usd_kes_data.shape}")
print(f"Date Range: {usd_kes_data['Date'].min()} to {usd_kes_data['Date'].max()}")
print(f"\nSample:")
print(usd_kes_data.head())



 All libraries imported successfully!
Pandas version: 2.2.1
NumPy version: 1.26.4
STOCK PRICE DATA LOADED

KCB Data Shape: (1306, 2)
Date Range: 2020-08-26 00:00:00 to 2025-11-21 00:00:00

First 5 rows:
        Date   Close
0 2020-08-26   35.05
1 2020-08-27   36.05
2 2020-08-28   36.40
3 2020-08-31   37.00
4 2020-09-01   37.95


Equity Bank Data Shape: (1306, 2)
Date Range: 2020-08-26 00:00:00 to 2025-11-21 00:00:00


Co-op Bank Data Shape: (1306, 2)
Date Range: 2020-08-26 00:00:00 to 2025-11-21 00:00:00

INTEREST RATE DATA

Original shape: (188, 7)
Missing values before imputation:
YEAR                        0
MONTH                       0
91-Day Tbill                0
182-days Tbill              0
364-days Tbill              0
Cash Reserve Requirement    0
Central Bank Rate           0
dtype: int64

Missing values after imputation:
YEAR                        0
MONTH                       0
91-Day Tbill                0
182-days Tbill              0
364-days Tbill              0
Cas

In [8]:


# Calculate daily returns: (Price_today - Price_yesterday) / Price_yesterday
kcb_data.columns = kcb_data.columns.str.strip()
equity_data.columns = equity_data.columns.str.strip()
coop_data.columns = coop_data.columns.str.strip()

kcb_data['Returns'] = kcb_data['Close'].pct_change()
equity_data['Returns'] = equity_data['Close'].pct_change()
coop_data['Returns'] = coop_data['Close'].pct_change()

# Remove the first NaN value from returns calculation
kcb_returns = kcb_data['Returns'].dropna()
equity_returns = equity_data['Returns'].dropna()
coop_returns = coop_data['Returns'].dropna()

print("=" * 70)
print("DAILY RETURNS CALCULATED")
print("=" * 70)

# Summary statistics
print("\n KCB RETURNS STATISTICS:")
print(f"Mean Daily Return: {kcb_returns.mean():.6f} ({kcb_returns.mean()*100:.4f}%)")
print(f"Std Deviation: {kcb_returns.std():.6f} ({kcb_returns.std()*100:.4f}%)")
print(f"Min Return: {kcb_returns.min():.6f} ({kcb_returns.min()*100:.2f}%)")
print(f"Max Return: {kcb_returns.max():.6f} ({kcb_returns.max()*100:.2f}%)")

print("\n EQUITY BANK RETURNS STATISTICS:")
print(f"Mean Daily Return: {equity_returns.mean():.6f} ({equity_returns.mean()*100:.4f}%)")
print(f"Std Deviation: {equity_returns.std():.6f} ({equity_returns.std()*100:.4f}%)")
print(f"Min Return: {equity_returns.min():.6f} ({equity_returns.min()*100:.2f}%)")
print(f"Max Return: {equity_returns.max():.6f} ({equity_returns.max()*100:.2f}%)")

print("\n CO-OP BANK RETURNS STATISTICS:")
print(f"Mean Daily Return: {coop_returns.mean():.6f} ({coop_returns.mean()*100:.4f}%)")
print(f"Std Deviation: {coop_returns.std():.6f} ({coop_returns.std()*100:.4f}%)")
print(f"Min Return: {coop_returns.min():.6f} ({coop_returns.min()*100:.2f}%)")
print(f"Max Return: {coop_returns.max():.6f} ({coop_returns.max()*100:.2f}%)")



DAILY RETURNS CALCULATED

 KCB RETURNS STATISTICS:
Mean Daily Return: 0.000611 (0.0611%)
Std Deviation: 0.017145 (1.7145%)
Min Return: -0.097525 (-9.75%)
Max Return: 0.095588 (9.56%)

 EQUITY BANK RETURNS STATISTICS:
Mean Daily Return: 0.000569 (0.0569%)
Std Deviation: 0.014482 (1.4482%)
Min Return: -0.091847 (-9.18%)
Max Return: 0.085382 (8.54%)

 CO-OP BANK RETURNS STATISTICS:
Mean Daily Return: 0.000690 (0.0690%)
Std Deviation: 0.014855 (1.4855%)
Min Return: -0.113879 (-11.39%)
Max Return: 0.099668 (9.97%)


In [9]:

# Create interactive plot with Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=kcb_data['Date'], 
    y=kcb_data['Close'],
    name='KCB',
    mode='lines',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=equity_data['Date'], 
    y=equity_data['Close'],
    name='Equity Bank',
    mode='lines',
    line=dict(color='green', width=2)
))

fig.add_trace(go.Scatter(
    x=coop_data['Date'], 
    y=coop_data['Close'],
    name='Co-op Bank',
    mode='lines',
    line=dict(color='red', width=2)
))

fig.update_layout(
    title='Kenyan Bank Stock Prices (Aug 2020 - Present)',
    xaxis_title='Date',
    yaxis_title='Price (KES)',
    hovermode='x unified',
    template='plotly_white',
    height=500,
    legend=dict(x=0.01, y=0.99)
)

fig.show()

# VaR Analysis for Kenyan Banks
# Part 2: Value-at-Risk Calculations Using 3 Methods


This section computes **three major VaR methods** widely used in financial risk management:



## **Historical VaR**
Historical VaR uses *actual past daily returns* to estimate potential future losses.

### How it works:
- Calculate daily returns from historical price data.
- Sort returns from worst to best.
- Pick the percentile corresponding to the confidence level:
  - 95% VaR → 5th percentile  
  - 99% VaR → 1st percentile  

This method makes **no distribution assumptions**, making it simple and realistic.



## **Parametric (Variance–Covariance) VaR**
This method assumes returns are **normally distributed**.

Formula:
VaR = z_alpha × sigma × Portfolio Value

Where:

sigma = daily volatility

z_alpha = critical value (1.65 for 95%, 2.33 for 99%)

This approach is fast, analytical, and widely used in reporting under normal market conditions.

## Monte Carlo Simulation VaR

Monte Carlo VaR generates thousands of hypothetical future price paths based on volatility and random shocks.

Process:

Assume returns follow a stochastic process.

Simulate thousands of random returns using:
r = mu + sigma * epsilon, epsilon ~ N(0,1)

Compute simulated portfolio values.

Take the worst 1–5% of outcomes.


Monte Carlo is the **most flexible** method and captures complex risk dynamics beyond the normal distribution.


In [10]:


def variance_covariance_var(returns, confidence_level=0.95, investment=1000000):
    """
    Calculate VaR using Variance-Covariance (Parametric) Method
    
    Parameters:
    - returns: array of daily returns
    - confidence_level: confidence level (default 95%)
    - investment: portfolio value in KES
    
    Returns:
    - VaR value in KES
    """
    mean = returns.mean()
    std = returns.std()
    
    # Z-score for confidence level
    z_score = norm.ppf(1 - confidence_level)
    
    # VaR calculation
    var = investment * (mean + z_score * std)
    
    return abs(var)


def historical_simulation_var(returns, confidence_level=0.95, investment=1000000):
    """
    Calculate VaR using Historical Simulation Method
    
    Parameters:
    - returns: array of daily returns
    - confidence_level: confidence level (default 95%)
    - investment: portfolio value in KES
    
    Returns:
    - VaR value in KES
    """
    # Find the percentile corresponding to (1 - confidence_level)
    percentile = (1 - confidence_level) * 100
    var_return = np.percentile(returns, percentile)
    
    # VaR calculation
    var = investment * abs(var_return)
    
    return var


def monte_carlo_var(returns, confidence_level=0.95, investment=1000000, 
                   num_simulations=10000):
    """
    Calculate VaR using Monte Carlo Simulation Method
    
    Parameters:
    - returns: array of daily returns
    - confidence_level: confidence level (default 95%)
    - investment: portfolio value in KES
    - num_simulations: number of Monte Carlo simulations
    
    Returns:
    - VaR value in KES
    - simulated_returns: array of simulated returns (for visualization)
    """
    mean = returns.mean()
    std = returns.std()
    
    # Generate random returns using normal distribution
    simulated_returns = np.random.normal(mean, std, num_simulations)
    
    # Find the percentile
    percentile = (1 - confidence_level) * 100
    var_return = np.percentile(simulated_returns, percentile)
    
    # VaR calculation
    var = investment * abs(var_return)
    
    return var, simulated_returns


print("VaR calculation functions defined!")
print("\nAvailable functions:")
print("  1. variance_covariance_var()")
print("  2. historical_simulation_var()")
print("  3. monte_carlo_var()")

VaR calculation functions defined!

Available functions:
  1. variance_covariance_var()
  2. historical_simulation_var()
  3. monte_carlo_var()


In [14]:

# Calculate VaR for KCB (Single Stock Analysis)


# Investment amount
initial_investment = 1000000  # 1 Million KES

# Confidence levels
confidence_levels = [0.95, 0.99]

print("=" * 70)
print("VALUE-AT-RISK ANALYSIS: KCB BANK")
print("=" * 70)
print(f"\nInitial Investment: KES {initial_investment:,.0f}")
print(f"Number of observations: {len(kcb_returns)}")
print(f"Time period: {kcb_data['Date'].min().date()} to {kcb_data['Date'].max().date()}")

# Store results
kcb_var_results = {}

for conf in confidence_levels:
    print(f"\n{'='*70}")
    print(f"CONFIDENCE LEVEL: {conf*100}%")
    print(f"{'='*70}")
    
    # Method 1: Variance-Covariance
    var_vc = variance_covariance_var(kcb_returns, conf, initial_investment)
    print(f"\n1. Variance-Covariance VaR: KES {var_vc:,.2f}")
    print(f"   Interpretation: {conf*100}% confident we won't lose more than KES {var_vc:,.2f} in one day")
    
    # Method 2: Historical Simulation
    var_hs = historical_simulation_var(kcb_returns, conf, initial_investment)
    print(f"\n2. Historical Simulation VaR: KES {var_hs:,.2f}")
    print(f"   Interpretation: Based on historical data, {conf*100}% of days had losses less than KES {var_hs:,.2f}")
    
    # Method 3: Monte Carlo Simulation
    var_mc, simulated = monte_carlo_var(kcb_returns, conf, initial_investment)
    print(f"\n3. Monte Carlo Simulation VaR: KES {var_mc:,.2f}")
    print(f"   Interpretation: Based on 10,000 simulations, {conf*100}% chance loss won't exceed KES {var_mc:,.2f}")
    
    # Store results
    kcb_var_results[f'{int(conf*100)}%'] = {
        'Variance-Covariance': var_vc,
        'Historical Simulation': var_hs,
        'Monte Carlo': var_mc
    }

VALUE-AT-RISK ANALYSIS: KCB BANK

Initial Investment: KES 1,000,000
Number of observations: 1305
Time period: 2020-08-26 to 2025-11-21

CONFIDENCE LEVEL: 95.0%

1. Variance-Covariance VaR: KES 27,590.78
   Interpretation: 95.0% confident we won't lose more than KES 27,590.78 in one day

2. Historical Simulation VaR: KES 22,603.99
   Interpretation: Based on historical data, 95.0% of days had losses less than KES 22,603.99

3. Monte Carlo Simulation VaR: KES 27,690.95
   Interpretation: Based on 10,000 simulations, 95.0% chance loss won't exceed KES 27,690.95

CONFIDENCE LEVEL: 99.0%

1. Variance-Covariance VaR: KES 39,275.13
   Interpretation: 99.0% confident we won't lose more than KES 39,275.13 in one day

2. Historical Simulation VaR: KES 47,692.46
   Interpretation: Based on historical data, 99.0% of days had losses less than KES 47,692.46

3. Monte Carlo Simulation VaR: KES 38,415.50
   Interpretation: Based on 10,000 simulations, 99.0% chance loss won't exceed KES 38,415.50


In [15]:

# Calculate VaR for All Three Banks


print("\n" + "=" * 70)
print("VALUE-AT-RISK ANALYSIS: ALL THREE BANKS")
print("=" * 70)

# Store all results
all_var_results = {}

banks = {
    'KCB': kcb_returns,
    'Equity Bank': equity_returns,
    'Co-op Bank': coop_returns
}

for bank_name, returns in banks.items():
    print(f"\n{'='*70}")
    print(f"{bank_name.upper()}")
    print(f"{'='*70}")
    
    bank_results = {}
    
    for conf in confidence_levels:
        var_vc = variance_covariance_var(returns, conf, initial_investment)
        var_hs = historical_simulation_var(returns, conf, initial_investment)
        var_mc, _ = monte_carlo_var(returns, conf, initial_investment)
        
        bank_results[f'{int(conf*100)}%'] = {
            'Variance-Covariance': var_vc,
            'Historical Simulation': var_hs,
            'Monte Carlo': var_mc
        }
        
        print(f"\nConfidence Level: {conf*100}%")
        print(f"  Variance-Covariance: KES {var_vc:,.2f}")
        print(f"  Historical Simulation: KES {var_hs:,.2f}")
        print(f"  Monte Carlo: KES {var_mc:,.2f}")
    
    all_var_results[bank_name] = bank_results



VALUE-AT-RISK ANALYSIS: ALL THREE BANKS

KCB

Confidence Level: 95.0%
  Variance-Covariance: KES 27,590.78
  Historical Simulation: KES 22,603.99
  Monte Carlo: KES 28,063.07

Confidence Level: 99.0%
  Variance-Covariance: KES 39,275.13
  Historical Simulation: KES 47,692.46
  Monte Carlo: KES 39,317.31

EQUITY BANK

Confidence Level: 95.0%
  Variance-Covariance: KES 23,251.62
  Historical Simulation: KES 21,085.83
  Monte Carlo: KES 23,334.41

Confidence Level: 99.0%
  Variance-Covariance: KES 33,120.91
  Historical Simulation: KES 37,784.74
  Monte Carlo: KES 33,336.70

CO-OP BANK

Confidence Level: 95.0%
  Variance-Covariance: KES 23,744.11
  Historical Simulation: KES 17,970.58
  Monte Carlo: KES 23,427.70

Confidence Level: 99.0%
  Variance-Covariance: KES 33,867.51
  Historical Simulation: KES 36,116.12
  Monte Carlo: KES 34,213.51


In [16]:

# Create Comparison Tables


print("\n" + "=" * 80)
print("VAR COMPARISON TABLE - 95% CONFIDENCE LEVEL")
print("=" * 80)

# Create DataFrame for 95% confidence
comparison_95 = pd.DataFrame({
    'Bank': ['KCB', 'Equity Bank', 'Co-op Bank'],
    'Variance-Covariance': [
        all_var_results['KCB']['95%']['Variance-Covariance'],
        all_var_results['Equity Bank']['95%']['Variance-Covariance'],
        all_var_results['Co-op Bank']['95%']['Variance-Covariance']
    ],
    'Historical Simulation': [
        all_var_results['KCB']['95%']['Historical Simulation'],
        all_var_results['Equity Bank']['95%']['Historical Simulation'],
        all_var_results['Co-op Bank']['95%']['Historical Simulation']
    ],
    'Monte Carlo': [
        all_var_results['KCB']['95%']['Monte Carlo'],
        all_var_results['Equity Bank']['95%']['Monte Carlo'],
        all_var_results['Co-op Bank']['95%']['Monte Carlo']
    ]
})

print("\n", comparison_95.to_string(index=False))

print("\n" + "=" * 80)
print("VAR COMPARISON TABLE - 99% CONFIDENCE LEVEL")
print("=" * 80)

# Create DataFrame for 99% confidence
comparison_99 = pd.DataFrame({
    'Bank': ['KCB', 'Equity Bank', 'Co-op Bank'],
    'Variance-Covariance': [
        all_var_results['KCB']['99%']['Variance-Covariance'],
        all_var_results['Equity Bank']['99%']['Variance-Covariance'],
        all_var_results['Co-op Bank']['99%']['Variance-Covariance']
    ],
    'Historical Simulation': [
        all_var_results['KCB']['99%']['Historical Simulation'],
        all_var_results['Equity Bank']['99%']['Historical Simulation'],
        all_var_results['Co-op Bank']['99%']['Historical Simulation']
    ],
    'Monte Carlo': [
        all_var_results['KCB']['99%']['Monte Carlo'],
        all_var_results['Equity Bank']['99%']['Monte Carlo'],
        all_var_results['Co-op Bank']['99%']['Monte Carlo']
    ]
})

print("\n", comparison_99.to_string(index=False))



VAR COMPARISON TABLE - 95% CONFIDENCE LEVEL

        Bank  Variance-Covariance  Historical Simulation  Monte Carlo
        KCB           27590.7775             22603.9852   28063.0710
Equity Bank           23251.6216             21085.8320   23334.4102
 Co-op Bank           23744.1076             17970.5788   23427.6959

VAR COMPARISON TABLE - 99% CONFIDENCE LEVEL

        Bank  Variance-Covariance  Historical Simulation  Monte Carlo
        KCB           39275.1311             47692.4628   39317.3109
Equity Bank           33120.9102             37784.7376   33336.6989
 Co-op Bank           33867.5086             36116.1180   34213.5115


In [17]:

# Visualize VaR Comparison (95% Confidence)


# Create bar chart comparing methods
fig = go.Figure()

methods = ['Variance-Covariance', 'Historical Simulation', 'Monte Carlo']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

for i, method in enumerate(methods):
    fig.add_trace(go.Bar(
        name=method,
        x=['KCB', 'Equity Bank', 'Co-op Bank'],
        y=comparison_95[method],
        marker_color=colors[i],
        text=[f'KES {x:,.0f}' for x in comparison_95[method]],
        textposition='outside'
    ))

fig.update_layout(
    title='VaR Comparison Across Banks (95% Confidence Level)',
    xaxis_title='Bank',
    yaxis_title='VaR (KES)',
    barmode='group',
    template='plotly_white',
    height=500,
    showlegend=True,
    legend=dict(x=0.7, y=0.99)
)

fig.show()

In [18]:

# Visualize VaR by Method for Each Bank


fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('KCB', 'Equity Bank', 'Co-op Bank')
)

banks_list = ['KCB', 'Equity Bank', 'Co-op Bank']

for idx, bank in enumerate(banks_list, 1):
    fig.add_trace(
        go.Bar(
            name='95% VaR',
            x=methods,
            y=[
                all_var_results[bank]['95%']['Variance-Covariance'],
                all_var_results[bank]['95%']['Historical Simulation'],
                all_var_results[bank]['95%']['Monte Carlo']
            ],
            marker_color='lightblue',
            showlegend=(idx == 1)
        ),
        row=1, col=idx
    )
    
    fig.add_trace(
        go.Bar(
            name='99% VaR',
            x=methods,
            y=[
                all_var_results[bank]['99%']['Variance-Covariance'],
                all_var_results[bank]['99%']['Historical Simulation'],
                all_var_results[bank]['99%']['Monte Carlo']
            ],
            marker_color='darkblue',
            showlegend=(idx == 1)
        ),
        row=1, col=idx
    )

fig.update_layout(
    title_text='VaR by Method: 95% vs 99% Confidence',
    height=500,
    template='plotly_white',
    barmode='group'
)

fig.show()

# VaR Analysis for Kenyan Banks
# Part 3: Portfolio VaR and Geometric Brownian Motion (GBM)


This section expands the VaR analysis beyond single assets by computing **portfolio-level VaR** and simulating price dynamics using **Geometric Brownian Motion**.


## **Portfolio VaR**
Instead of looking at each bank’s stock individually, we compute the risk of a **combined portfolio**.

### What this includes:
- Computing weighted portfolio returns
- Estimating portfolio volatility
- Calculating Historical, Parametric, and Monte Carlo VaR for the entire basket

Portfolio VaR accounts for diversification and correlations between assets, providing a more realistic measure of overall risk exposure.

In [19]:

# Create Portfolio Returns


print("=" * 70)
print("PORTFOLIO CONSTRUCTION")
print("=" * 70)

# Portfolio weights (equal weight for simplicity, but can be adjusted)
weights = {
    'KCB': 0.40,
    'Equity Bank': 0.30,
    'Co-op Bank': 0.30
}

print(f"\nPortfolio Composition:")
print(f"  KCB: {weights['KCB']*100}%")
print(f"  Equity Bank: {weights['Equity Bank']*100}%")
print(f"  Co-op Bank: {weights['Co-op Bank']*100}%")

# Align all returns by date (use common dates)
# Merge dataframes on date
kcb_df = kcb_data[['Date', 'Returns']].rename(columns={'Returns': 'KCB_Returns'})
equity_df = equity_data[['Date', 'Returns']].rename(columns={'Returns': 'Equity_Returns'})
coop_df = coop_data[['Date', 'Returns']].rename(columns={'Returns': 'Coop_Returns'})

# Merge all returns
portfolio_df = kcb_df.merge(equity_df, on='Date', how='inner')
portfolio_df = portfolio_df.merge(coop_df, on='Date', how='inner')

# Remove NaN values
portfolio_df = portfolio_df.dropna()

# Calculate portfolio returns as weighted average
portfolio_df['Portfolio_Returns'] = (
    weights['KCB'] * portfolio_df['KCB_Returns'] +
    weights['Equity Bank'] * portfolio_df['Equity_Returns'] +
    weights['Co-op Bank'] * portfolio_df['Coop_Returns']
)

portfolio_returns = portfolio_df['Portfolio_Returns']

print(f"\nPortfolio Statistics:")
print(f"  Number of observations: {len(portfolio_returns)}")
print(f"  Mean daily return: {portfolio_returns.mean():.6f} ({portfolio_returns.mean()*100:.4f}%)")
print(f"  Std deviation: {portfolio_returns.std():.6f} ({portfolio_returns.std()*100:.4f}%)")
print(f"  Min return: {portfolio_returns.min():.6f} ({portfolio_returns.min()*100:.2f}%)")
print(f"  Max return: {portfolio_returns.max():.6f} ({portfolio_returns.max()*100:.2f}%)")

# Calculate annualized metrics
trading_days = 252
annualized_return = portfolio_returns.mean() * trading_days
annualized_volatility = portfolio_returns.std() * np.sqrt(trading_days)
sharpe_ratio = annualized_return / annualized_volatility  # Assuming risk-free rate ≈ 0 for simplicity

print(f"\nAnnualized Metrics:")
print(f"  Annualized Return: {annualized_return*100:.2f}%")
print(f"  Annualized Volatility: {annualized_volatility*100:.2f}%")
print(f"  Sharpe Ratio: {sharpe_ratio:.4f}")


PORTFOLIO CONSTRUCTION

Portfolio Composition:
  KCB: 40.0%
  Equity Bank: 30.0%
  Co-op Bank: 30.0%

Portfolio Statistics:
  Number of observations: 1305
  Mean daily return: 0.000622 (0.0622%)
  Std deviation: 0.010870 (1.0870%)
  Min return: -0.052730 (-5.27%)
  Max return: 0.059702 (5.97%)

Annualized Metrics:
  Annualized Return: 15.67%
  Annualized Volatility: 17.26%
  Sharpe Ratio: 0.9081


In [20]:

# Calculate Portfolio VaR


print("\n" + "=" * 70)
print("PORTFOLIO VAR ANALYSIS")
print("=" * 70)

initial_investment = 1000000
portfolio_var_results = {}

for conf in [0.95, 0.99]:
    print(f"\n{'='*70}")
    print(f"CONFIDENCE LEVEL: {conf*100}%")
    print(f"{'='*70}")
    
    var_vc = variance_covariance_var(portfolio_returns, conf, initial_investment)
    var_hs = historical_simulation_var(portfolio_returns, conf, initial_investment)
    var_mc, _ = monte_carlo_var(portfolio_returns, conf, initial_investment)
    
    portfolio_var_results[f'{int(conf*100)}%'] = {
        'Variance-Covariance': var_vc,
        'Historical Simulation': var_hs,
        'Monte Carlo': var_mc
    }
    
    print(f"\nVariance-Covariance VaR: KES {var_vc:,.2f}")
    print(f"Historical Simulation VaR: KES {var_hs:,.2f}")
    print(f"Monte Carlo VaR: KES {var_mc:,.2f}")
    
    # Calculate average VaR
    avg_var = np.mean([var_vc, var_hs, var_mc])
    print(f"\nAverage VaR: KES {avg_var:,.2f}")
    print(f"As % of portfolio: {(avg_var/initial_investment)*100:.2f}%")


PORTFOLIO VAR ANALYSIS

CONFIDENCE LEVEL: 95.0%

Variance-Covariance VaR: KES 17,257.42
Historical Simulation VaR: KES 15,001.35
Monte Carlo VaR: KES 17,196.45

Average VaR: KES 16,485.07
As % of portfolio: 1.65%

CONFIDENCE LEVEL: 99.0%

Variance-Covariance VaR: KES 24,665.12
Historical Simulation VaR: KES 28,037.40
Monte Carlo VaR: KES 24,183.77

Average VaR: KES 25,628.76
As % of portfolio: 2.56%


In [ ]:

# CELL 15: Compare Individual Stocks vs Portfolio VaR


print("\n" + "=" * 70)
print("PORTFOLIO DIVERSIFICATION BENEFIT")
print("=" * 70)

# Calculate weighted average of individual VaRs (95% confidence)
weighted_individual_var = (
    weights['KCB'] * all_var_results['KCB']['95%']['Monte Carlo'] +
    weights['Equity Bank'] * all_var_results['Equity Bank']['95%']['Monte Carlo'] +
    weights['Co-op Bank'] * all_var_results['Co-op Bank']['95%']['Monte Carlo']
)

portfolio_var_mc = portfolio_var_results['95%']['Monte Carlo']

diversification_benefit = weighted_individual_var - portfolio_var_mc
diversification_benefit_pct = (diversification_benefit / weighted_individual_var) * 100

print(f"\n95% Monte Carlo VaR Comparison:")
print(f"  Weighted Avg of Individual Stocks: KES {weighted_individual_var:,.2f}")
print(f"  Portfolio VaR: KES {portfolio_var_mc:,.2f}")
print(f"  Diversification Benefit: KES {diversification_benefit:,.2f}")
print(f"  Benefit as %: {diversification_benefit_pct:.2f}%")
print(f"\n  Interpretation: By holding a diversified portfolio,")
print(f"  risk is reduced by {diversification_benefit_pct:.2f}% compared to concentrated positions")


PORTFOLIO DIVERSIFICATION BENEFIT

95% Monte Carlo VaR Comparison:
  Weighted Avg of Individual Stocks: KES 25,189.51
  Portfolio VaR: KES 17,543.43
  Diversification Benefit: KES 7,646.08
  Benefit as %: 30.35%

  Interpretation: By holding a diversified portfolio,
  risk is reduced by 30.35% compared to concentrated positions




## **Geometric Brownian Motion (GBM) Simulation**
GBM is a common model for stock prices under the assumption that returns follow a lognormal distribution.


### What we do here:
- Simulate future price paths for a bank's stock
- Visualize possible future trajectories
- Compute VaR using these GBM-generated scenarios

This technique mimics future uncertainty and highlights potential extreme outcomes beyond what historical data alone can show.


In [21]:

# Geometric Brownian Motion (GBM) Implementation


print("\n" + "=" * 70)
print("GEOMETRIC BROWNIAN MOTION (GBM) SIMULATION")
print("=" * 70)

def gbm_simulation(S0, mu, sigma, T, dt, num_simulations=10000):
    """
    Simulate stock prices using Geometric Brownian Motion
    
    Parameters:
    - S0: Initial stock price
    - mu: Expected return (drift)
    - sigma: Volatility
    - T: Time horizon in years
    - dt: Time step (e.g., 1/252 for daily)
    - num_simulations: Number of simulation paths
    
    Returns:
    - Simulated final prices
    """
    num_steps = int(T / dt)
    
    # Generate random shocks
    random_shocks = np.random.normal(0, 1, (num_simulations, num_steps))
    
    # Initialize price array
    prices = np.zeros((num_simulations, num_steps + 1))
    prices[:, 0] = S0
    
    # Simulate prices using GBM formula
    for t in range(1, num_steps + 1):
        prices[:, t] = prices[:, t-1] * np.exp(
            (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * random_shocks[:, t-1]
        )
    
    return prices


def gbm_monte_carlo_var(S0, mu, sigma, T, confidence_level=0.95, 
                       investment=1000000, num_simulations=10000):
    """
    Calculate VaR using GBM-based Monte Carlo simulation
    
    Parameters:
    - S0: Current stock price
    - mu: Expected return (drift)
    - sigma: Volatility
    - T: Time horizon (e.g., 1/252 for 1 day)
    - confidence_level: Confidence level
    - investment: Portfolio value
    - num_simulations: Number of simulations
    
    Returns:
    - VaR value
    - simulated final prices
    """
    dt = T  # For single period
    
    # Simulate final prices
    final_prices = S0 * np.exp(
        (mu - 0.5 * sigma**2) * T + 
        sigma * np.sqrt(T) * np.random.normal(0, 1, num_simulations)
    )
    
    # Calculate returns
    returns = (final_prices - S0) / S0
    
    # Find VaR
    percentile = (1 - confidence_level) * 100
    var_return = np.percentile(returns, percentile)
    var = investment * abs(var_return)
    
    return var, final_prices, returns


# Example: GBM VaR for KCB
S0_kcb = kcb_data['Close'].iloc[-1]  # Current price
mu_kcb = kcb_returns.mean() * 252  # Annualized return
sigma_kcb = kcb_returns.std() * np.sqrt(252)  # Annualized volatility
T = 1/252  # 1 day

print(f"\nKCB GBM Parameters:")
print(f"  Current Price (S0): KES {S0_kcb:.2f}")
print(f"  Expected Annual Return (μ): {mu_kcb*100:.2f}%")
print(f"  Annual Volatility (σ): {sigma_kcb*100:.2f}%")
print(f"  Time Horizon: 1 trading day")

var_gbm_95, prices_gbm, returns_gbm = gbm_monte_carlo_var(
    S0_kcb, mu_kcb, sigma_kcb, T, 0.95, initial_investment, 10000
)

var_gbm_99, _, _ = gbm_monte_carlo_var(
    S0_kcb, mu_kcb, sigma_kcb, T, 0.99, initial_investment, 10000
)

print(f"\nGBM-Based Monte Carlo VaR for KCB:")
print(f"  95% VaR: KES {var_gbm_95:,.2f}")
print(f"  99% VaR: KES {var_gbm_99:,.2f}")

# Compare with basic Monte Carlo
var_basic_95 = all_var_results['KCB']['95%']['Monte Carlo']
var_basic_99 = all_var_results['KCB']['99%']['Monte Carlo']

print(f"\nComparison with Basic Monte Carlo:")
print(f"  Basic MC (95%): KES {var_basic_95:,.2f}")
print(f"  GBM-MC (95%): KES {var_gbm_95:,.2f}")
print(f"  Difference: KES {abs(var_gbm_95 - var_basic_95):,.2f}")


GEOMETRIC BROWNIAN MOTION (GBM) SIMULATION

KCB GBM Parameters:
  Current Price (S0): KES 64.25
  Expected Annual Return (μ): 15.39%
  Annual Volatility (σ): 27.22%
  Time Horizon: 1 trading day

GBM-Based Monte Carlo VaR for KCB:
  95% VaR: KES 27,023.55
  99% VaR: KES 38,320.83

Comparison with Basic Monte Carlo:
  Basic MC (95%): KES 28,063.07
  GBM-MC (95%): KES 27,023.55
  Difference: KES 1,039.52


In [ ]:

# Visualize GBM Simulation Paths


# Simulate multiple price paths for visualization
num_paths = 100
T_horizon = 21  # 21 trading days (1 month)
dt = 1/252

prices_paths = gbm_simulation(S0_kcb, mu_kcb, sigma_kcb, T_horizon/252, dt, num_paths)

# Create time array
time_array = np.arange(0, T_horizon + 1)

# Plot simulated paths
fig = go.Figure()

# Plot sample paths
for i in range(min(50, num_paths)):  # Plot first 50 paths
    fig.add_trace(go.Scatter(
        x=time_array,
        y=prices_paths[i, :],
        mode='lines',
        line=dict(width=0.5, color='lightblue'),
        showlegend=False,
        hoverinfo='skip'
    ))

# Add mean path
mean_path = prices_paths.mean(axis=0)
fig.add_trace(go.Scatter(
    x=time_array,
    y=mean_path,
    mode='lines',
    name='Mean Path',
    line=dict(width=3, color='darkblue')
))

# Add confidence intervals
percentile_5 = np.percentile(prices_paths, 5, axis=0)
percentile_95 = np.percentile(prices_paths, 95, axis=0)

fig.add_trace(go.Scatter(
    x=time_array,
    y=percentile_95,
    mode='lines',
    name='95th Percentile',
    line=dict(width=2, color='green', dash='dash')
))

fig.add_trace(go.Scatter(
    x=time_array,
    y=percentile_5,
    mode='lines',
    name='5th Percentile',
    line=dict(width=2, color='red', dash='dash')
))

fig.update_layout(
    title='KCB Stock Price Simulation using GBM (21 Trading Days)',
    xaxis_title='Trading Days',
    yaxis_title='Price (KES)',
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()


# Visualize GBM Returns Distribution


fig = go.Figure()

# Histogram of simulated returns
fig.add_trace(go.Histogram(
    x=returns_gbm * 100,
    nbinsx=50,
    name='Simulated Returns',
    marker_color='lightblue',
    opacity=0.7
))

# Add VaR line
var_95_pct = (var_gbm_95 / initial_investment) * 100
fig.add_vline(
    x=-var_95_pct,
    line_dash="dash",
    line_color="red",
    annotation_text=f"95% VaR: -{var_95_pct:.2f}%",
    annotation_position="top"
)

fig.update_layout(
    title='Distribution of Simulated Returns (GBM-Based Monte Carlo)',
    xaxis_title='Return (%)',
    yaxis_title='Frequency',
    template='plotly_white',
    height=500,
    showlegend=True
)

fig.show()



# VaR Analysis for Kenyan Banks - Financial Risk Management Project
## Part 4: Black-Scholes Option Pricing and Comprehensive Comparison


This section introduces the Black–Scholes (B–S) analytical formula for European option pricing, explains the assumptions and inputs, and describes how option valuations feed into portfolio VaR and stress testing.


### What is the Black–Scholes model?
The Black–Scholes model is an analytical framework to price **European call and put options** under the assumption that the underlying asset follows a Geometric Brownian Motion with constant volatility. It provides closed-form expressions for option prices and is widely used as a baseline model in risk management.

### Key assumptions
- Underlying asset price follows Geometric Brownian Motion (lognormal returns).
- Constant volatility (σ) and constant risk-free rate (r).
- No dividends (or continuous dividend yield q if included).
- Markets are frictionless (no transaction costs, no arbitrage).
- Options are European (exercise only at maturity).


###  Black–Scholes formulas (European options)

Define:
\[
d_1 = \frac{\ln\!\left(\tfrac{S}{K}\right) + \left(r - q + \tfrac{1}{2}\sigma^2\right)T}{\sigma\sqrt{T}},\qquad
d_2 = d_1 - \sigma\sqrt{T}
\]

Where:
- \(S\) = current spot price of underlying  
- \(K\) = option strike price  
- \(r\) = continuously compounded risk-free rate  
- \(q\) = continuous dividend yield (use 0 if none)  
- \(\sigma\) = volatility (annualised)  
- \(T\) = time to maturity (in years)  

**Call price:**
\[
C = S e^{-qT}\Phi(d_1) - K e^{-rT}\Phi(d_2)
\]

**Put price:**
\[
P = K e^{-rT}\Phi(-d_2) - S e^{-qT}\Phi(-d_1)
\]

where \(\Phi(\cdot)\) is the standard normal cumulative distribution function.



###  Inputs
- **Spot price (S):** most recent market price in your dataset.  
- **Strike (K):** option strike; for vanilla options choose a set of strikes to test moneyness.  
- **Time to maturity (T):** expressed in years (e.g., 30 days → 30/252 ≈ 0.119). Use trading days convention you’ve used elsewhere.  
- **Volatility (σ):** could be historical volatility (from returns) or implied volatility if available. For scenario analysis consider stress-scaled volatilities (e.g., +50% vol).  
- **Risk-free rate (r):** use an appropriate short-term rate for Kenya (or proxies) consistent with your VaR horizon.  
- **Dividend yield (q):** if the underlying pays dividends; otherwise set q = 0.


###  Why include Black–Scholes in VaR analysis?
- Option positions have **nonlinear payoffs** (convexity) that standard linear VaR methods understate.  
- Use B–S to value options at current spot and under simulated future spots (for Monte Carlo):  
  - In Monte Carlo VaR, simulate underlying paths → price options at each path using the B–S formula (with appropriate T and σ) → compute portfolio P&L distribution.  
- B–S also provides **deltas** (and other Greeks) that allow delta-approximate VaR (linearization) or delta–gamma approximations for more accurate parametric VaR.


###  Implementation notes (to match notebook & `risk_management_var.py`)
- Implement a `black_scholes_price(S, K, T, r, sigma, q=0, option_type='call')` function that returns `C` or `P`.  
- Implement helper `d1_d2(S,K,T,r,sigma,q)` that computes `d1` and `d2`.  
- For Monte Carlo scenarios, compute option values at each simulated terminal spot (or at intermediate times for path-dependent analysis) and include option P&L when constructing the simulated portfolio returns.  
- For stress tests, re-price options under stressed volatility and interest rate scenarios to capture nonlinear effects on capital shortfalls.


###  Short pointers for reporting
- State the source of volatility (historical vs implied) and the T convention used (calendar vs trading days).  
- If you use delta-approximation for parametric VaR, document the approximations and justify them.  
- Save Black–Scholes pricing helper functions in `risk_management_var.py` (the repository contains the notebook and the `risk_management_var.py` file referenced earlier).


In [25]:
# VaR Analysis for Kenyan Banks - Financial Risk Management Project
# Part 4: Black-Scholes Option Pricing and Comprehensive Comparison


# Black-Scholes Option Pricing Functions


from scipy.stats import norm

def black_scholes_call(S, K, T, r, sigma):
    """
    Calculate European Call Option Price using Black-Scholes Formula
    
    Parameters:
    - S: Current stock price
    - K: Strike price
    - T: Time to expiration (in years)
    - r: Risk-free interest rate (annual)
    - sigma: Volatility (annual)
    
    Returns:
    - Call option price
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
    return call_price


def black_scholes_put(S, K, T, r, sigma):
    """
    Calculate European Put Option Price using Black-Scholes Formula
    
    Parameters:
    - S: Current stock price
    - K: Strike price
    - T: Time to expiration (in years)
    - r: Risk-free interest rate (annual)
    - sigma: Volatility (annual)
    
    Returns:
    - Put option price
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return put_price


def calculate_greeks(S, K, T, r, sigma, option_type='call'):
    """
    Calculate option Greeks
    
    Returns:
    - Delta, Gamma, Theta, Vega, Rho
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        delta = norm.cdf(d1)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) 
                 - r * K * np.exp(-r * T) * norm.cdf(d2))
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)
    else:  # put
        delta = norm.cdf(d1) - 1
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) 
                 + r * K * np.exp(-r * T) * norm.cdf(-d2))
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T)
    
    return {'delta': delta, 'gamma': gamma, 'theta': theta, 'vega': vega, 'rho': rho}


print("=" * 70)
print("BLACK-SCHOLES OPTION PRICING MODEL")
print("=" * 70)
print("\n✓ Black-Scholes functions defined!")
print("\nAvailable functions:")
print("  1. black_scholes_call()")
print("  2. black_scholes_put()")
print("  3. calculate_greeks()")


BLACK-SCHOLES OPTION PRICING MODEL

✓ Black-Scholes functions defined!

Available functions:
  1. black_scholes_call()
  2. black_scholes_put()
  3. calculate_greeks()


In [ ]:

# Portfolio Insurance Using Protective Puts (KCB Example)


print("\n" + "=" * 70)
print("PORTFOLIO INSURANCE ANALYSIS: KCB BANK")
print("=" * 70)

# Current parameters
S_kcb = kcb_data['Close'].iloc[-1]  # Current price
sigma_kcb_annual = kcb_returns.std() * np.sqrt(252)  # Annual volatility

# Get risk-free rate from CBK data (use latest CBR)
# Assuming CBR is around 13% (Kenya's typical rate)
r_annual = 0.13  # 13% annual rate

# Portfolio details
num_shares = initial_investment / S_kcb  # Number of shares in portfolio
print(f"\nPortfolio Details:")
print(f"  Current Stock Price: KES {S_kcb:.2f}")
print(f"  Number of Shares: {num_shares:.2f}")
print(f"  Portfolio Value: KES {initial_investment:,.0f}")
print(f"  Annual Volatility: {sigma_kcb_annual*100:.2f}%")
print(f"  Risk-Free Rate: {r_annual*100:.2f}%")

# Calculate protective put prices for different protection levels and time horizons
protection_levels = [0.90, 0.95, 0.97]  # 90%, 95%, 97% of current price
time_horizons = [30, 60, 90]  # days

print("\n" + "=" * 70)
print("PROTECTIVE PUT OPTION PRICING")
print("=" * 70)

put_prices_df = []

for protection in protection_levels:
    K = S_kcb * protection  # Strike price
    
    for days in time_horizons:
        T = days / 365  # Convert to years
        
        # Calculate put price
        put_price = black_scholes_put(S_kcb, K, T, r_annual, sigma_kcb_annual)
        total_cost = put_price * num_shares
        cost_pct = (total_cost / initial_investment) * 100
        
        put_prices_df.append({
            'Protection Level': f'{int(protection*100)}%',
            'Strike Price (KES)': K,
            'Time Horizon (days)': days,
            'Put Price per Share (KES)': put_price,
            'Total Cost (KES)': total_cost,
            'Cost as % of Portfolio': cost_pct
        })
        
        print(f"\nProtection at {protection*100:.0f}% of current price (K = KES {K:.2f})")
        print(f"  Time: {days} days")
        print(f"  Put Price: KES {put_price:.2f} per share")
        print(f"  Total Cost: KES {total_cost:,.2f}")
        print(f"  Cost as % of Portfolio: {cost_pct:.2f}%")

# Create DataFrame
put_prices_table = pd.DataFrame(put_prices_df)

print("\n" + "=" * 70)
print("PUT OPTION PRICING TABLE")
print("=" * 70)
print(put_prices_table.to_string(index=False))



PORTFOLIO INSURANCE ANALYSIS: KCB BANK

Portfolio Details:
  Current Stock Price: KES 64.25
  Number of Shares: 15564.20
  Portfolio Value: KES 1,000,000
  Annual Volatility: 27.22%
  Risk-Free Rate: 13.00%

PROTECTIVE PUT OPTION PRICING

Protection at 90% of current price (K = KES 57.83)
  Time: 30 days
  Put Price: KES 0.14 per share
  Total Cost: KES 2,220.38
  Cost as % of Portfolio: 0.22%

Protection at 90% of current price (K = KES 57.83)
  Time: 60 days
  Put Price: KES 0.41 per share
  Total Cost: KES 6,446.04
  Cost as % of Portfolio: 0.64%

Protection at 90% of current price (K = KES 57.83)
  Time: 90 days
  Put Price: KES 0.65 per share
  Total Cost: KES 10,166.42
  Cost as % of Portfolio: 1.02%

Protection at 95% of current price (K = KES 61.04)
  Time: 30 days
  Put Price: KES 0.59 per share
  Total Cost: KES 9,181.12
  Cost as % of Portfolio: 0.92%

Protection at 95% of current price (K = KES 61.04)
  Time: 60 days
  Put Price: KES 1.05 per share
  Total Cost: KES 16,288

In [28]:
# Compare VaR Loss vs Hedging Cost


print("\n" + "=" * 70)
print("COST-BENEFIT ANALYSIS: VAR LOSS vs HEDGING COST")
print("=" * 70)

# Focus on 30-day horizon with 95% protection
var_95_kcb = all_var_results['KCB']['95%']['Monte Carlo']
put_cost_30d_95 = put_prices_table[
    (put_prices_table['Protection Level'] == '95%') & 
    (put_prices_table['Time Horizon (days)'] == 30)
]['Total Cost (KES)'].values[0]

print(f"\n30-Day Risk Analysis:")
print(f"  95% VaR (Potential 1-day Loss): KES {var_95_kcb:,.2f}")
print(f"  Cost to Hedge (30-day put at 95%): KES {put_cost_30d_95:,.2f}")
print(f"  Ratio (Hedging Cost / VaR): {(put_cost_30d_95 / var_95_kcb):.2f}x")

if put_cost_30d_95 < var_95_kcb:
    print(f"\n   RECOMMENDATION: Hedging is cost-effective")
    print(f"    Paying KES {put_cost_30d_95:,.2f} to protect against potential KES {var_95_kcb:,.2f} loss")
    savings_pct = ((var_95_kcb - put_cost_30d_95) / var_95_kcb) * 100
    print(f"    Potential savings: {savings_pct:.1f}% compared to taking the loss")
else:
    print(f"\n  ⚠ CONSIDERATION: Hedging costs exceed 1-day VaR")
    print(f"    May be suitable for risk-averse portfolios or longer horizons")



COST-BENEFIT ANALYSIS: VAR LOSS vs HEDGING COST

30-Day Risk Analysis:
  95% VaR (Potential 1-day Loss): KES 28,063.07
  Cost to Hedge (30-day put at 95%): KES 9,181.12
  Ratio (Hedging Cost / VaR): 0.33x

   RECOMMENDATION: Hedging is cost-effective
    Paying KES 9,181.12 to protect against potential KES 28,063.07 loss
    Potential savings: 67.3% compared to taking the loss


In [29]:
# Visualize Put Option Prices

# Create heatmap of put prices
pivot_table = put_prices_table.pivot_table(
    values='Cost as % of Portfolio',
    index='Protection Level',
    columns='Time Horizon (days)'
)

fig = go.Figure(data=go.Heatmap(
    z=pivot_table.values,
    x=pivot_table.columns,
    y=pivot_table.index,
    colorscale='Blues',
    text=pivot_table.values,
    texttemplate='%{text:.2f}%',
    textfont={"size": 12},
    colorbar=dict(title="Cost as %<br>of Portfolio")
))

fig.update_layout(
    title='Put Option Cost as % of Portfolio Value',
    xaxis_title='Time Horizon (days)',
    yaxis_title='Protection Level',
    height=400,
    template='plotly_white'
)

fig.show()

# Bar chart comparing different protection levels (30-day horizon)
fig = go.Figure()

data_30d = put_prices_table[put_prices_table['Time Horizon (days)'] == 30]

fig.add_trace(go.Bar(
    x=data_30d['Protection Level'],
    y=data_30d['Total Cost (KES)'],
    text=[f'KES {x:,.0f}' for x in data_30d['Total Cost (KES)']],
    textposition='outside',
    marker_color='lightblue'
))

# Add VaR line for comparison
fig.add_hline(
    y=var_95_kcb,
    line_dash="dash",
    line_color="red",
    annotation_text=f"95% VaR: KES {var_95_kcb:,.0f}",
    annotation_position="right"
)

fig.update_layout(
    title='30-Day Put Option Cost vs 1-Day VaR (KCB)',
    xaxis_title='Protection Level',
    yaxis_title='Cost (KES)',
    template='plotly_white',
    height=500
)

fig.show()

In [30]:
#Comprehensive Comparison Table - All Methods

print("\n" + "=" * 80)
print("COMPREHENSIVE RISK METRICS COMPARISON - KCB BANK")
print("=" * 80)

comprehensive_comparison = pd.DataFrame({
    'Risk Measure': [
        'Variance-Covariance VaR (95%)',
        'Historical Simulation VaR (95%)',
        'Basic Monte Carlo VaR (95%)',
        'GBM Monte Carlo VaR (95%)',
        '30-Day Put Option (95% Strike)',
        '30-Day Put Option (90% Strike)',
        'Variance-Covariance VaR (99%)',
        'GBM Monte Carlo VaR (99%)'
    ],
    'Value (KES)': [
        all_var_results['KCB']['95%']['Variance-Covariance'],
        all_var_results['KCB']['95%']['Historical Simulation'],
        all_var_results['KCB']['95%']['Monte Carlo'],
        var_gbm_95,
        put_prices_table[(put_prices_table['Protection Level'] == '95%') & 
                        (put_prices_table['Time Horizon (days)'] == 30)]['Total Cost (KES)'].values[0],
        put_prices_table[(put_prices_table['Protection Level'] == '90%') & 
                        (put_prices_table['Time Horizon (days)'] == 30)]['Total Cost (KES)'].values[0],
        all_var_results['KCB']['99%']['Variance-Covariance'],
        var_gbm_99
    ],
    'Type': [
        'Loss Estimate', 'Loss Estimate', 'Loss Estimate', 'Loss Estimate',
        'Hedging Cost', 'Hedging Cost', 'Loss Estimate', 'Loss Estimate'
    ]
})

print("\n", comprehensive_comparison.to_string(index=False))


COMPREHENSIVE RISK METRICS COMPARISON - KCB BANK

                    Risk Measure  Value (KES)          Type
  Variance-Covariance VaR (95%)   27590.7775 Loss Estimate
Historical Simulation VaR (95%)   22603.9852 Loss Estimate
    Basic Monte Carlo VaR (95%)   28063.0710 Loss Estimate
      GBM Monte Carlo VaR (95%)   27023.5498 Loss Estimate
 30-Day Put Option (95% Strike)    9181.1244  Hedging Cost
 30-Day Put Option (90% Strike)    2220.3757  Hedging Cost
  Variance-Covariance VaR (99%)   39275.1311 Loss Estimate
      GBM Monte Carlo VaR (99%)   38320.8265 Loss Estimate


In [31]:
# Final Visualization - All Methods Compared

# Separate VaR and Option prices
var_data = comprehensive_comparison[comprehensive_comparison['Type'] == 'Loss Estimate']
option_data = comprehensive_comparison[comprehensive_comparison['Type'] == 'Hedging Cost']

fig = go.Figure()

# VaR methods
fig.add_trace(go.Bar(
    name='VaR Methods (Potential Loss)',
    x=var_data['Risk Measure'],
    y=var_data['Value (KES)'],
    marker_color='lightcoral',
    text=[f'KES {x:,.0f}' for x in var_data['Value (KES)']],
    textposition='outside'
))

# Hedging costs
fig.add_trace(go.Bar(
    name='Put Option Hedging Cost',
    x=option_data['Risk Measure'],
    y=option_data['Value (KES)'],
    marker_color='lightgreen',
    text=[f'KES {x:,.0f}' for x in option_data['Value (KES)']],
    textposition='outside'
))

fig.update_layout(
    title='Comprehensive Risk Analysis: VaR Methods vs Option Hedging (KCB)',
    xaxis_title='Risk Measure',
    yaxis_title='Amount (KES)',
    template='plotly_white',
    height=600,
    barmode='group',
    xaxis_tickangle=-45
)

fig.show()